In [ ]:
!nvidia-smi
!fuser -v /dev/nvidia0

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
!ls

drive  sample_data


In [35]:
import os
import cv2
import math
import random
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB4
import albumentations

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
AUTO = tf.data.experimental.AUTOTUNE


2.5.0


In [5]:
class CFG:
    seed = 54
    classes = 11014 
    scale = 30 
    margin = 0.5
    fc_dim = 512
    img_size = 380  #对应EfficientNetB4的输入分辨率 
    batch_size = 8
    channels = 3

In [6]:
def read_dataset(csv_path, image_path):
    df = pd.read_csv(csv_path)
    image_paths = image_path + df['image']
    return df, image_paths

In [7]:
image_path = '/content/drive/MyDrive/shopee/shopee-product-matching/test_images/'
csv_path = '/content/drive/MyDrive/shopee/shopee-product-matching/test.csv' # repalce the train.csv
df, image_paths = read_dataset(csv_path, image_path)

In [8]:
df.head()

,posting_id,image,image_phash,title
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng


In [9]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [10]:
def get_image_neighbors(df, embeddings, KNN=50):

    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    threshold = 4.5
    predictions = []
    for k in tqdm(range(embeddings.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = df['posting_id'].iloc[ids].values
        predictions.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    return df, predictions

In [11]:
# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [12]:

class EffModel(Model):
    def __init__(self, n_classes = CFG.classes,
        img_size = CFG.img_size,
        fc_dim = CFG.fc_dim
        ):
        super(EffModel, self).__init__()
        
        inputs = layers.Input(shape=(img_size, img_size, 3))
        #inputs = albumentations(inputs)
        self.backbone = EfficientNetB4(include_top=False, input_tensor=inputs, weights="imagenet")
        
        # Freeze the pretrained weights
        # self.backbone.trainable = False
        
        self.pooling = layers.GlobalAveragePooling2D(name="avg_pool")
        self.flatten = layers.Flatten()
        
        # Rebuild top
        #self.dropout = layers.Dropout(rate = 0.1, name="dropout")
        self.classifier = layers.Dense(fc_dim, activation="relu", name="pred") # rule
        #self.bn = layers.BatchNormalization()      

    def call(self, inputs):
        features = self.extract_features(inputs) 
        return features
    
    def extract_features(self, x):
        # effnet + pooling + fc
        x = self.backbone(x)
        x = self.pooling(x)   
        x = self.flatten(x) 

        #x = self.dropout(x)
        x = self.classifier(x)
        # x = self.bn(x)
            
        return x

In [13]:
# Function to decode our images
def preprocess_image(image):
    
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  
    image = tf.image.resize(image, [CFG.img_size, CFG.img_size])
    return image

# Function to read our test image and return image
def load_and_preprocess_image(image, label):
    image = tf.io.read_file(image)
    image = preprocess_image(image)
    return image, label

def get_dataset(image, label):
    dataset = tf.data.Dataset.from_tensor_slices((image, label))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls = AUTO) 
    # dataset = dataset.map(data_augment, num_parallel_calls=AUTO) # data_augment
    return dataset


In [28]:

# 1. datasets
dataset = get_dataset(image_paths, tf.zeros(shape=len(image_paths)))

# split the datasets
test_ds = dataset.cache(filename='./cache.tf-data').shuffle(CFG.batch_size*20).batch(CFG.batch_size).prefetch(AUTO)

In [33]:
def create_model():

    backbone = EffModel()

    margin = ArcMarginProduct(n_classes = CFG.classes, 
                  s = 30, 
                  m = 0.5, 
                  name='head/arc_margin', 
                  dtype='float32'
                  )

    image = tf.keras.layers.Input(shape=(CFG.img_size, CFG.img_size, 3), name='input/image')
    label = tf.keras.layers.Input((), name='input/label')

    x = backbone(image) 
    x = margin([x, label]) # Archead

    model = tf.keras.models.Model(inputs = [image, label], outputs = x)

    return model

In [36]:
reconstructed_model = create_model()
reconstructed_model.load_weights(saved_model_path)
# 重新取输入和输出 选取embedding层作为输出
reconstructed_model = tf.keras.models.Model(inputs = reconstructed_model.input[0], outputs = reconstructed_model.layers[-3].output) 
reconstructed_model.build((None, 380, 380, 3))
reconstructed_model.summary()

71688192/71686520 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input/image (InputLayer)     [(None, 380, 380, 3)]     0         
_________________________________________________________________
eff_model_3 (EffModel)       (None, 512)               18593887  
Total params: 18,593,887
Trainable params: 919,040
Non-trainable params: 17,674,847
_________________________________________________________________


In [ ]:
embeds = []
for images, labels in tqdm(test_ds):
      embeds.append(reconstructed_model.predict(images))

100%|██████████| 172/172 [00:40<00:00,  4.20it/s]


In [ ]:
def get_text_predictions(df, max_features=25000):
    
    model = TfidfVectorizer(stop_words='english', binary=True, max_features=max_features)
    text_embeddings = model.fit_transform(df['title']).toarray()

    print('Finding similar titles...')
    CHUNK = 1024 * 4
    CTS = len(df) // CHUNK
    if (len(df)%CHUNK) != 0:
        CTS += 1

    preds = []
    for j in range( CTS ):
        a = j * CHUNK
        b = (j+1) * CHUNK
        b = min(b, len(df))
        print('chunk', a, 'to', b)

        # COSINE SIMILARITY DISTANCE
        cts = np.matmul(text_embeddings, text_embeddings[a:b].T).T
        for k in range(b-a):
            IDX = np.where(cts[k,]>0.75)[0]
            o = df.iloc[np.asarray(IDX)].posting_id.values
            preds.append(o)

    del model,text_embeddings
    gc.collect()
    return preds

In [ ]:
text_predictions = get_text_predictions(df, max_features=25000)

Finding similar titles...
chunk 0 to 3425


In [ ]:
image_embeddings = np.concatenate(embeds)
df, image_predictions = get_image_neighbors(df, image_embeddings, KNN = 3) # 

100%|██████████| 3425/3425 [00:00<00:00, 9704.10it/s]


In [ ]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions']])
    return ' '.join( np.unique(x) )

In [ ]:
df['image_predictions'] = image_predictions
df['text_predictions'] = text_predictions
df['matches'] = df.apply(combine_predictions, axis=1)
df[['posting_id', 'matches']].to_csv('/content/drive/MyDrive/shopee/outputs/submission.csv', index=False)

In [ ]:
df['f1'] = f1_score(df['posting_id'], df['matches'])
df['f1'].mean()

In [ ]:
df[['posting_id','image_predictions', 'text_predictions', 'matches', 'f1']].head()

,posting_id,image_predictions,text_predictions,matches,f1
912,train_192154327,"[train_192154327, train_4249618609, train_1532...",[train_192154327],train_153214025 train_192154327 train_4249618609,0.5
24857,train_3602971377,"[train_3602971377, train_2834821897, train_223...",[train_3602971377],train_2231906914 train_2834821897 train_360297...,0.5
7276,train_4280407716,"[train_4280407716, train_181310095, train_1182...",[train_4280407716],train_1182906906 train_181310095 train_4280407716,0.5
21442,train_898978553,"[train_898978553, train_750786421, train_38098...",[train_898978553],train_3809881145 train_750786421 train_898978553,0.5
8190,train_2680742165,"[train_2680742165, train_3540642385, train_977...",[train_2680742165],train_2680742165 train_3540642385 train_977743751,0.5
